# 人工智能AI岗位全国城市区域分布以及薪资分布可视化

## 1、读取爬取结果，查看基本信息

### 1.1 导包并定义常量

In [1]:
import pandas as pd
import re

In [2]:
DATA_PATH = 'data/data.csv' # 定义路径常量
PICTURE_BASE_PATH ='html/' 

### 1.2 查看数据信息

基本信息说明：job-职位名称|
city-工作城市|
company-公司名称|
people-公司规模|
salary-薪水|
experience-工作经验|
education-教育背景|
skill-要求技能  
读取数据

In [3]:
data = pd.read_csv(DATA_PATH, dtype = 'str')

查看前5条数据

In [4]:
data.head()

,job,city,company,people,salary,experience,education,skill
0,AI无人直播（苏州独家）代理商,苏州,微服文化传媒,20-99人,11-22K,经验不限,学历不限,['品牌招商']
1,初中数学文本解析老师,苏州,自贡一快教育,20-99人,40-45元/时,经验不限,大专,"['兼职', '高薪', '自由']"
2,AI录音师（兼职）,苏州,道泰信息,20-99人,6-7K,经验不限,学历不限,"['声音剪辑', '声音设计', '现场录音']"
3,数据标注/AI训练师,苏州,苏州艾星仕信息科技,20-99人,25-30元/时,经验不限,学历不限,"['兼职工作', '数据标注', '图片标注', '语音标注', '视频标注', '区域标注..."
4,AI专员,苏州,茂禾地产,100-499人,5-10K·13薪,1-3年,大专,"['语音/图像识别', 'Python']"


可以看出：  
1、salary薪资的形式有很多种，需要统一。  
2、job列“初中数学文本解析老师”与目标职业不符，需要对这类数据剔除。

使用describe()函数获取描述性统计数据

In [5]:
data.describe(include = 'all')

,job,city,company,people,salary,experience,education,skill
count,7797,7797,7797,7797,7797,7797,7797,7797
unique,4868,69,5989,7,817,9,9,6765
top,ai产品经理,北京,某500强上市公司,20-99人,10-15K,经验不限,本科,['人工智能']
freq,124,359,232,2328,364,2183,3895,46


统计结果：  
1、job、city、company、people、salary、experience、education、skill列count相同，无缺失值。  
2、job列总数庞大，共4868种不同的工作，需要进行数据清洗。  
3、近8000个city中有69个不同的城市，而最多一个城市出现359次，表明数据中有部分城市只出现少数次。意味着需要去除仅出现在“全国”而没有出现在“热门城市”的城市数据。  
4、people有7种，表明公司规模可以初步划分为7类。  
5、experience与education都为9，初步划分为9类。  
6、skill技能要求有近7000种，需要进行数据清洗。

In [6]:
print(set(data['experience'])) # 对experience列查看
print(set(data['education'])) # 对education列查看
print(set(data['people'])) # 对people列查看

{'1-3年', '3-5年', '1年以内', '10年以上', 'experience', '在校/应届', '5-10年', '经验不限', '应届生'}
{'本科', '高中', '大专', '学历不限', '博士', '硕士', '初中及以下', '中专/中技', 'education'}
{'20-99人', '100-499人', '500-999人', 'people', '1000-9999人', '0-20人', '10000人以上'}


发现experience、education、people初步划分不合适，需要对experience、education、people整合分类

## 2、数据预处理

### 2.1 薪资salary清洗

#### 2.2.1 清洗数据

定义薪资清洗函数

In [7]:
def calculate_monthly_salary(salary_str):  
    # 先匹配“25-40K·13薪”型
    if re.search(r'\d{1,3}-\d{1,3}K·\d+薪', salary_str):  
        parts = re.findall(r'\d+', salary_str)  # 找到所有的数字部分  
        monthly_salary = int(int(parts[0]) + int(parts[1])) /2 * 1000   #12-19K
        months = int(parts[2])  # 15薪  
        return monthly_salary * months / 12 
        
    # 匹配"6-11K"型
    elif re.search(r'\d{1,3}-\d{1,3}K', salary_str):  
        return (int(re.findall(r'\d+', salary_str)[0]) + int(re.findall(r'\d+',salary_str)[1]))/2 * 1000 
        
    # 匹配"8K"型
    elif re.search(r'^\d{1,3}K$', salary_str): 
        return  int(re.findall(r'\d+', salary_str)[0]) * 1000

    # 舍弃“30-50元/时，100-200元/天，5000-4000元/月”型
    else:  
        return None  

对salary列清洗

In [8]:
data['salary'] = data['salary'].apply(calculate_monthly_salary)
data.dropna(axis = 0, inplace = True) # 去除空行

#### 2.2.2 查看清洗结果

In [9]:
data['salary']

0       16500.0
2        6500.0
4        8125.0
5       11375.0
6        5625.0
         ...   
7792     8000.0
7793     4000.0
7794     5000.0
7795     6500.0
7796    10500.0
Name: salary, Length: 7670, dtype: float64

### 2.2 经验experience、学历education、公司规模people清洗

#### 2.2.1 清洗数据

替换在校、应届生为不满一年

In [10]:
data['experience'] = data['experience'].replace({'1年以内':'不满1年','在校/应届':'不满1年','应届生':'不满1年'})

替换学历

In [11]:
data['education'] = data['education'].replace({'初中及以下':'高中及以下','高中':'高中及以下','中专/中技':'专科','大专':'专科'})

替换公司规模

In [12]:
data['people'] = data['people'].replace({'0-20人':'100人以下','20-99人':'100人以下'})

#### 2.2.2 查看清洗结果

In [13]:
data.head(5)

,job,city,company,people,salary,experience,education,skill
0,AI无人直播（苏州独家）代理商,苏州,微服文化传媒,100人以下,16500.0,经验不限,学历不限,['品牌招商']
2,AI录音师（兼职）,苏州,道泰信息,100人以下,6500.0,经验不限,学历不限,"['声音剪辑', '声音设计', '现场录音']"
4,AI专员,苏州,茂禾地产,100-499人,8125.0,1-3年,专科,"['语音/图像识别', 'Python']"
5,AI应用研发工程师(J10500),苏州,华硕科技（苏州）...,1000-9999人,11375.0,不满1年,本科,"['业务导向', '语音/图像识别', '机器学习算法/工程化经验']"
6,AI数据标注工程师,苏州,苏州科达,1000-9999人,5625.0,经验不限,本科,['图片标注']


## 3、数据分析与绘图

### 3.1 定义绘图函数

#### 3.1.1 导包

In [14]:
import pyecharts.options as opts
from pyecharts.charts import Pie
from pyecharts.charts import Bar
from pyecharts.charts import Boxplot
from pyecharts.charts import WordCloud
from pyecharts.charts import Bar3D
from pyecharts.charts import Map
from IPython.display import IFrame #用于在jupyter notebook中显示html

#### 3.1.2 定义各种图形函数

In [15]:
# 饼图
def pie(label:list, values:list, title:str, html_name:str):
    item = tuple(zip(label, values))
    data = sorted(item, key = lambda x: x[1], reverse = True)
    label = [i[0] for i in data]
    values = [i[1] for i in data]
    
    pie = (
        Pie(init_opts=opts.InitOpts(bg_color="#2c343c"))
        .add(
            "",
            [list(z) for z in zip(label, values)],
            rosetype="radius",
            radius=["35%","75%"], # 设置内外半圆半径
            center=["50%", "50%"], # 设置图形居中

        )  
        .set_series_opts(
            label_opts = opts.LabelOpts(formatter = "{b} \n {d}%"),
        )   
        .set_global_opts(
            title_opts=opts.TitleOpts(
            title=title,
            pos_left="20",
            pos_bottom="20",
            title_textstyle_opts=opts.TextStyleOpts(color="rgba(255, 255, 255, 0.5"),
            ),
            # Legend设置 
            legend_opts = opts.LegendOpts(background_color="rgba(255, 255, 255, 0.5",border_width = 0)
        )
    )
    pie.render(PICTURE_BASE_PATH + html_name),

In [16]:
# 箱线图
def box(label:list, values:list, title:str, html_name:str):
    box = Boxplot(init_opts=opts.InitOpts(bg_color="#2c343c"))
    box.add_xaxis(label)
    box.add_yaxis("", box.prepare_data(values))
    box.set_global_opts(
        title_opts=opts.TitleOpts(
            title=title, pos_left="20", pos_bottom="20",title_textstyle_opts=opts.TextStyleOpts(color="rgba(255, 255, 255, 0.5")
        )
    )  
    box.render(PICTURE_BASE_PATH + html_name)

In [17]:
# 词云图
def wordCloud(pair, title:str, html_name:str):
    word_cloud = (
        WordCloud(init_opts=opts.InitOpts(bg_color="#2c343c"))
        .add(series_name = "", data_pair = pair, word_size_range=[10, 100])
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title=title,
                pos_left="20",
                pos_bottom="20",
                title_textstyle_opts=opts.TextStyleOpts(color="rgba(255, 255, 255, 0.5")
            ),
        )
        .render(PICTURE_BASE_PATH + html_name)
    )

In [18]:
# 3d柱状图
def three(grouped, result:list, title:str, html_name:str):
    x = list(set(grouped['experience']))
    y = list(set(grouped['education']))
    
    three = (
        Bar3D()
        .add(
            "",
            result,
            xaxis3d_opts=opts.Axis3DOpts(x, type_="category"),
            yaxis3d_opts=opts.Axis3DOpts(y, type_="category"),
            zaxis3d_opts=opts.Axis3DOpts(type_="value"),
        )
        .set_global_opts(
            visualmap_opts=opts.VisualMapOpts(max_=120000, pos_right = "10"),
            title_opts=opts.TitleOpts(
                title=title, pos_left="0", pos_bottom="40",title_textstyle_opts=opts.TextStyleOpts(color="rgba(0, 0, 0, 0.5")
            )
        )
        .render(PICTURE_BASE_PATH + html_name)
    )

In [19]:
# 地图
def map(map_lst:list, title:str, html_name:str, max:int):
    """
    :param: map_lst
    """
    map = (
        Map(init_opts=opts.InitOpts(bg_color="#2c343c"))
        .add(
            "",
            map_lst,
            "china-cities",
            label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title=title,
                pos_left="20",
                pos_bottom="40",
                title_textstyle_opts=opts.TextStyleOpts(color="rgba(255, 255, 255, 0.5")
            ),
            visualmap_opts=opts.VisualMapOpts(max_=max,pos_right = "10"),
            # visualmap_opts=opts.VisualMapOpts(pos_right = '10'),
        )
        .render(PICTURE_BASE_PATH + html_name)
    )


### 3.2 岗位数与经验、学历、公司规模的关系

#### 3.1.1 岗位数与经验

In [20]:
people_df = data['experience'].value_counts().reset_index()
label = list(people_df['experience'])
values = list(people_df['count'])

pie(label, values, '岗位数-经验', 'experience.html')
IFrame(src = PICTURE_BASE_PATH + 'experience.html',width = 1000,height = 600)

经验要求不限、在1-3年、3-5年的比重可谓三权分立。  
相比于卷了几十年的软件工程等计算机专业其他分流，AI起步较晚，对经验要求较低。

#### 3.1.2 岗位数与学历

In [21]:
# 将计数系列转换为数据框，以便查找每个值对应的 'education'  
education_df = data['education'].value_counts().reset_index()
label = list(education_df['education'])
values = list(education_df['count'])

pie(label, values, '岗位数-学历','jobCount_education.html')
IFrame(src = PICTURE_BASE_PATH + 'jobCount_education.html',width = 1000,height = 600)

本科可以从事AI工作的岗位数量占比超过了50%，说明本科毕业仍然能找到一份AI相关的工作。

#### 3.1.3 岗位数与公司规模

In [22]:
people_df = data['people'].value_counts().reset_index()
label = list(people_df['people'])
values = list(people_df['count'])

pie(label, values, '岗位数-公司规模', 'companyPeople.html')
IFrame(src = PICTURE_BASE_PATH + 'companyPeople.html',width = 1000,height = 600)

公司规模在100人以下的占比接近50%，说明AI相关岗位有很多初创公司、小型公司。

### 3.3 岗位薪酬与经验、学历、公司规模的关系

#### 3.2.1 薪酬与经验

In [23]:
# 不统计经验不限
experience_groups = data.groupby('experience')['salary'].apply(list)  
# 将分组结果合并为一个列表  
experience_salary_list = experience_groups.values.tolist()
experience_list = experience_groups.index.tolist()
box(experience_list, experience_salary_list, '岗位薪水-经验','experience_salary.html')
IFrame(src = PICTURE_BASE_PATH + 'experience_salary.html',width = 1200,height = 600)

无论经验在哪个阶段，都存在：  
少数极高工资能够达到30-50万/月；大部分处于较低水平，但仍然在1-2万/月之间。  
说明AI相关岗位薪水比较可观，并且随着经验的增加，薪水也总体呈上升趋势。

#### 3.2.2 薪酬与学历

In [24]:
# 按照教育程度分组，并计算每的薪水数据列表  
education_groups = data.groupby('education')['salary'].apply(list)  
# 将分组结果合并为一个列表  
education_salary_list = education_groups.values.tolist()
education_list = education_groups.index.tolist()
box(education_list, education_salary_list, '岗位薪水-学历','education_salary.html')
IFrame(src = PICTURE_BASE_PATH + 'education_salary.html',width = 1200,height = 600)

硕士、博士的平均薪水明显要高于其他学历水平，并且箱线图的分布范围也更大一点。  
说明虽然本科甚至专科就能找到一份AI工作，但是学历在硕士以上的薪水明显上了一个台阶，薪酬区间也更大一点。

#### 3.2.3 薪酬与公司规模

In [25]:
# 按照公司规模分组，并计算每的薪水数据列表  
people_groups = data.groupby('people')['salary'].apply(list)  
# 将分组结果合并为一个列表  
people_salary_list = people_groups.values.tolist()
people_list = people_groups.index.tolist()
box(people_list, people_salary_list, '岗位薪水-公司规模','people_salary.html')
IFrame(src = PICTURE_BASE_PATH + 'people_salary.html',width = 1200,height = 600)

岗位平均薪酬随着公司规模的增大而呈上升趋势。   
小公司的极大值可能还要比大型公司的极大值要高，也说明了小型公司的分化相对更加明显，而大型公司薪酬区间更稳定一些。

### 3.4岗位技能词云

In [26]:
# 提取技能列  
skill_lst = str([skill for skill in data['skill']])
skills_str = skill_lst.replace('[','').replace(']','').replace('"','')
# 将字符串 s 分割成独立的字串  
words = list(skills_str.split(','))
word_counts = {}  
for word in words:  
    if word in word_counts:  
        word_counts[word] += 1  
    else:  
        word_counts[word] = 1 
        
# 由于全部统计用时过长，所以去除出现次数小于30的词语       
new_word_counts = {}
for word, count in word_counts.items():  
    if count >= 30:  
        new_word_counts[word] = count
        
# 去除每个键两侧空白和单引号
new_word_counts = {key.strip().replace("'",""):value for key, value in new_word_counts.items()}
pair = list(new_word_counts.items())

In [27]:
#绘制词云
cloud = wordCloud(pair, '岗位技能词云','wordCloud.html')
IFrame(src = PICTURE_BASE_PATH + 'wordCloud.html',width = 1000,height = 600)

深度学习框架TensorFlow、PyTorch占比较大，并且机器学习算法/工程化经验、统计等数学知识的要求明显较高。  
在非互联网岗位如PS、机器人、销售等行业中AI也在逐渐渗透，表明AI在传统行业仍然有很好的前景。

### 3.5 岗位薪酬与经验和学历的三维关系

In [28]:
# 按experience、education分组并计算每组平均salary，重置索引
grouped = data.groupby(['experience', 'education'])['salary'].mean().reset_index() 
# 生成 “经验-学历-薪水” 对应三元组
result = grouped[['experience', 'education', 'salary']].values.tolist()
#绘制三维关系图
three(grouped, result, '岗位薪酬-经验/学历','three.html')
IFrame(src = PICTURE_BASE_PATH + 'three.html',width = 1000,height = 600)

可以看到，硕士博士的起步工资要高于本科专科，在5年以下博士的平均工资远远高于相同经验的其他学历的工资。  
具有5-10、10年以上经验的硕士、博士的工资水平遥遥领先。  
本科的工资随着经验的增加也有着不错的提升，而专科的薪酬提升就比较困难了。  

### 3.6 薪酬的城市分布

In [29]:
# 按city分组并计算每个city的平均salary，重置索引
grouped_city = data.groupby('city')['salary'].mean().reset_index()
# 生成 “城市-平均薪资” 对应列表
result_city_lst = grouped_city[['city','salary']].values.tolist()
# 添加城市的后缀“市”
result_city_lst = [[city[0] + '市',city[1]] for city in result_city_lst]
# print(result_city_lst)
map(result_city_lst, '薪酬的城市分布', 'map_city_salary.html', int(40000))
IFrame(src = PICTURE_BASE_PATH + 'map_city_salary.html',width = 1000,height = 800)

北京、上海一线城市，杭州、天津等沿海城市AI岗位薪酬比较高，在35-60万左右。   
武汉、重庆、西安、合肥等靠近中部地区AI岗位薪酬较为中等，在20-30万左右。  
太原、晋城等靠近中部偏北部的城市薪酬偏低，甚至许多薪酬没有过万，可能大多数岗位是需要AI的非互联网行业。